<a href="https://colab.research.google.com/github/LorenaGiVa/Caso---Marketing/blob/main/d_despliegue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Conectar el drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import openpyxl
import sys ## saber ruta de la que carga paquetes
sys.path ###Ruta directorio qué tiene paquetes
sys.path.append('/content/drive/MyDrive/Caso marketing analítica') ## este comanda agrega una ruta
import a_funciones as fn ## para procesamiento

In [3]:
####Paquete para sistema basado en contenido ####
from sklearn.preprocessing import MinMaxScaler
from sklearn import neighbors

In [46]:
def preprocesar():

    #### conectar_base_de_Datos#################
    conn=sql.connect('/content/drive/MyDrive/Caso marketing analítica/data/db_movies')
    cur=conn.cursor()



    ##### llevar datos que cambian constantemente a python ######
    movies=pd.read_sql('select * from movies', conn )
    ratings=pd.read_sql('select * from ratings', conn)
    usuarios=pd.read_sql('select distinct (user_id) as user_id from ratings',conn)
    #### transformación de datos crudos - Preprocesamiento ################
    movies['year_pub']=movies.year_pub.astype('int')
     ##### escalar para que año esté en el mismo rango ###
    sc=MinMaxScaler()
    movies[["year_sc"]]=sc.fit_transform(movies[['year_pub']])
    movies_dum1=movies.drop(columns=['user_id', 'movie_id', 'rating', 'movie_title', 'movie_genres', 'fecha_nueva'])
    col_dum=['movie_genres']
    movies_dum2=pd.get_dummies(movies_dum1,columns=col_dum)
    return movies_dum2,movies, conn, cur

In [47]:
##########################################################################
###############Función para entrenar modelo por cada usuario ##########
###############Basado en contenido todo lo visto por el usuario Knn#############################
def recomendar(user_id):

    movies_dum2, movies, conn, cur= preprocesar()
    ratings=pd.read_sql('select *from ratings where user_id=:user',conn, params={'user':user_id})
    l_movies_r=ratings['movie_id'].to_numpy()
    movies_dum2[['movie_id','movie_title']]=movies[['movie_id','movie_title']]
    movies_r=movies_dum2[movies_dum2['movie_id'].isin(l_movies_r)]
    movies_r=movies_r.drop(columns=['movie_id','movie_title'])
    movies_r["indice"]=1 ### para usar group by y que quede en formato pandas tabla de centroide
    centroide=movies_r.groupby("indice").mean()

    movies_nr=movies_dum2[~movies_dum2['movie_id'].isin(l_movies_r)]
    movies_nr=movies_nr.drop(columns=['movie_id','movie_title'])
    model=neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
    model.fit(movies_nr)
    dist, idlist = model.kneighbors(centroide)

    ids=idlist[0]
    recomend_b=movies.loc[ids][['book_title','isbn']]


    return recomend_b

In [48]:
##### Generar recomendaciones para usuario lista de usuarios ####
##### No se hace para todos porque es muy pesado #############
def main(list_user):

    recomendaciones_todos=pd.DataFrame()
    for user_id in list_user:

        recomendaciones=recomendar(user_id)
        recomendaciones["user_id"]=user_id
        recomendaciones.reset_index(inplace=True,drop=True)

        recomendaciones_todos=pd.concat([recomendaciones_todos, recomendaciones])

    recomendaciones_todos.to_excel('/content/drive/MyDrive/Caso marketing analítica/data/salidas/recomendaciones.xlsx')
    recomendaciones_todos.to_csv('/content/drive/MyDrive/Caso marketing analítica/data/salidas/recomendaciones.csv')

if __name__=="__main__":
    list_user=[430,200,105,10]
    main(list_user)


import sys
sys.executable

DatabaseError: Execution failed on sql 'select distinct (user_id) as user_id from ratings': no such column: user_id